<a href="https://colab.research.google.com/github/anishankar/anishankar.github.io/blob/master/MusicTranscriberDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#import libraries for nerualnet, math and visualization
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models
import scipy.io.wavfile as wav
import wave, glob, os, json
import librosa
import librosa.display
print(tf.__version__)

2.3.0


# Loading Test Note Data

In [3]:
from scipy.io.wavfile import read
mytest_audio = [];
path = '/content/drive/My Drive/TestNotes/'
for filename in glob.glob(os.path.join(path, '*.wav')):
  print(filename)
  y,s = librosa.core.load(filename, sr = 16000)
  mytest_audio.append(np.array(y*32000))

/content/drive/My Drive/TestNotes/TestE.wav
/content/drive/My Drive/TestNotes/TestC.wav
/content/drive/My Drive/TestNotes/TestC1.wav
/content/drive/My Drive/TestNotes/TestD.wav
/content/drive/My Drive/TestNotes/TestE2.wav
/content/drive/My Drive/TestNotes/TestC2.wav
/content/drive/My Drive/TestNotes/TestC3.wav
/content/drive/My Drive/TestNotes/TestC4.wav
/content/drive/My Drive/TestNotes/TestE3.wav
/content/drive/My Drive/TestNotes/TestE1.wav
/content/drive/My Drive/TestNotes/TestE4.wav
/content/drive/My Drive/TestNotes/TestD1.wav
/content/drive/My Drive/TestNotes/TestF.wav
/content/drive/My Drive/TestNotes/TestF1.wav
/content/drive/My Drive/TestNotes/TestF2.wav
/content/drive/My Drive/TestNotes/TestD2.wav
/content/drive/My Drive/TestNotes/TestG2.wav
/content/drive/My Drive/TestNotes/TestF3.wav
/content/drive/My Drive/TestNotes/TestG3.wav
/content/drive/My Drive/TestNotes/TestG.wav
/content/drive/My Drive/TestNotes/TestD3.wav
/content/drive/My Drive/TestNotes/TestG4.wav
/content/drive/

#Loading Neural Network Models

In [ ]:
model_cnn_mfcc = models.load_model('drive/My Drive/model_cnn_mfcc') 
model_cnn_chroma= models.load_model('drive/My Drive/model_cnn_chroma')

# Chroma Feature and MFCC Feature Calculation

In [5]:
mychroma_test = []
for i in range(len(mytest_audio)):
  temp = []
  for n in range(len(mytest_audio[i])//2048):
     temp.append(librosa.feature.chroma_stft(y=(mytest_audio[i][1024*n:1024*(n+2)]).astype(float), sr=16000))
  mychroma_test.append(np.array(temp).reshape(len(temp),12, 5))
for i in range(len(mychroma_test)):
  mychroma_test[i] = mychroma_test[i].reshape(mychroma_test[i].shape[0],12,5,1)

mymfcc_test = []
for i in range(len(mytest_audio)):
  temp = []
  for n in range(len(mytest_audio[i])//2048):
     temp.append(librosa.feature.mfcc(y=(mytest_audio[i][1024*n:1024*(n+2)]).astype(float), sr=16000, n_mfcc=30))
  mymfcc_test.append(np.array(temp).reshape(len(temp),30, 5)[:,2:30,:])
for i in range(len(mymfcc_test)):
  mymfcc_test[i] = mymfcc_test[i].reshape(mymfcc_test[i].shape[0],28,5,1)

# Testing on Solo Test Notes

In [34]:
#Test Note Recognition
number=0
wrong = 0
correct = [4, 0, 0, 2, 4, 0, 0, 0, 4, 4, 4, 2, 5, 5, 5, 2, 7, 5, 7, 7, 2, 7, 5, 7, 2]
          
for n in range(len(mychroma_test)):
  predict = model_cnn_chroma.predict(mychroma_test[n])
  pred_note = np.argmax(predict, axis = 1)
  number+= len(pred_note)
  for i in range(len(pred_note)):
    if pred_note[i] != correct[n]:
      wrong+=1
  print(pred_note)
print('Accuracy of ChromaCNN:')
print((number-wrong)/number)

[11  0  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  4  4  4  4]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[ 3  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  1  2  0
  0 10  0  0 11 11  6  6  4  4]
[5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
[5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
[5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
[3 2 2 2

In [28]:
#Test Octave Recognition
number=0
wrong = 0
correct = [2,3,3,4,2,3,3,3,2,2,2,4,4,4,4,4,3,4,3,3,4,3,4,3,4]
model_cnn_mfcc = models.load_model('drive/My Drive/model_cnn_mfcc')  

for n in range(len(mymfcc_test)):
  predict = model_cnn_mfcc.predict(mymfcc_test[n])
  pred_octave = np.argmax(predict, axis = 1)+1
  number+= len(pred_octave)
  for i in range(len(pred_octave)):
    if pred_octave[i] != correct[n]:
      wrong+=1
  print(pred_octave)
print('Accuracy of MFCC_CNN:')
print((number-wrong)/number)



Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f9d6c08e668> and <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f9d6c08ec88>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f9d6c08ec88> and <tensorflow.python.keras.layers.core.Flatten object at 0x7f9d6c093320>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x7f9d6c0937b8> and <tensorflow.python.keras.layers.core.Flatten object at 0x7f9d6c093320>).
[6 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[2 2 3 3 3 2 3 3 3 3 3]
[2 2 2 2 2 2 3 2 3 2 3]
[3 3 4 4 4 4 4 4 3 1 5 2 5 4 3 3 4 4 4 3 4 4 1 4 3 3 4 4 4 4 3 2 4 5 5 4 4
 5 4 1 4 4 1 4 4 4 4 4]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[2 2 2 2 2 2 2 2 3 2 2]
[2 2 2 2 3 2 2 3 2 3 2]
[2 2 2 2 2 2 2 3 3 3 3]
[

#Testing on Let It Be Demo Track


In [7]:
from scipy.io.wavfile import read
path = '/content/drive/My Drive/LetItBeTest.wav'
y,s = librosa.core.load(path, sr = 16000)
demo_audio = np.array(y*32000)

In [ ]:
from IPython.display import Audio
Audio(demo_audio,rate=16000)

In [13]:
#Chroma Calculation
demo_chroma = []
for n in range(len(demo_audio)//1024 - 1):
     demo_chroma.append(librosa.feature.chroma_stft(y=(demo_audio[1024*n:1024*(n+2)]).astype(float), sr=16000))
demo_chroma = np.array(demo_chroma).reshape(len(demo_chroma),12, 5)
demo_chroma = demo_chroma.reshape(demo_chroma.shape[0],12,5,1)

#actual = [A3,G3,E3,D3,C3,E3,G3,G3,E3,G3,G3,A3,C4,D4,E4,E4,D4,C4,D4,E4,E4,G4,E4,D4,C4,D4,E4,G4,E4,D4,E4,D4,C4,
#          A3,G3,E3,D3,C3,D3,E3,G3,A3,C4,A3,C4,D4,E4,G4,E4,G4,E4,G4,E4,G4,A4,A4,G4,A4,C5,E5,E5,E5,D5,C5,D5,C5,
#          D5,A5,E5,D5,C5,A4,G4,E4,D4,C4]

#Prediction
predict = model_cnn_chroma.predict(demo_chroma)

p = np.argmax(predict, axis = 1)
notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
perceived_notes = []
for i in p:
  perceived_notes.append(notes[i])

#bundle consecutive same notes together
curr_note = 'A'
count = 0
final_notes = []
for note in perceived_notes[10:]:
  if note == curr_note:
    count += 1
  else:
    final_notes.append(curr_note)
    count = 1
    curr_note = note
print(final_notes)

/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


['A', 'G', 'G#', 'E', 'C', 'D', 'C#', 'C', 'B', 'D', 'E', 'G#', 'G', 'G#', 'G', 'E', 'G#', 'G', 'G#', 'C', 'E', 'G', 'G#', 'A', 'C', 'A', 'D', 'D#', 'E', 'D', 'A', 'C', 'D', 'E', 'C', 'G#', 'F', 'E', 'A', 'D', 'C', 'D', 'E', 'G#', 'F', 'G#', 'G', 'C', 'E', 'C#', 'D', 'F#', 'E', 'C', 'A', 'E', 'D', 'A', 'C', 'D', 'D#', 'E', 'D#', 'E', 'G', 'A', 'A#', 'C', 'A', 'C', 'C#', 'B', 'D#', 'E', 'F', 'E', 'G#', 'F', 'G#', 'F', 'G', 'F', 'C', 'E', 'G#', 'F', 'G#', 'G', 'E', 'G#', 'G', 'F', 'G#', 'E', 'A', 'G', 'D', 'A', 'D#', 'E', 'A', 'E', 'F', 'E', 'A', 'D', 'C', 'F#', 'D', 'A', 'C', 'A', 'C', 'D', 'D#', 'G', 'D', 'A', 'D', 'C', 'A#', 'F', 'E', 'D', 'C', 'D', 'A', 'G', 'A', 'D', 'A#', 'C', 'D']


In [33]:
#MFCC calculation
demo_mfcc = []
for n in range(len(demo_audio)//1024 - 1):
     demo_mfcc.append(librosa.feature.mfcc(y=(demo_audio[1024*n:1024*(n+2)]).astype(float), sr=16000, n_mfcc = 30))
demo_mfcc = np.array(demo_mfcc).reshape(len(demo_mfcc),30, 5)
demo_mfcc = demo_mfcc[:,2:30,:]
demo_mfcc = demo_mfcc.reshape(demo_mfcc.shape[0],28,5,1)

#actual = [A3,G3,E3,D3,C3,E3,G3,G3,E3,G3,G3,A3,C4,D4,E4,E4,D4,C4,D4,E4,E4,G4,E4,D4,C4,D4,E4,G4,E4,D4,E4,D4,C4,
#          A3,G3,E3,D3,C3,D3,E3,G3,A3,C4,A3,C4,D4,E4,G4,E4,G4,E4,G4,E4,G4,A4,A4,G4,A4,C5,E5,E5,E5,D5,C5,D5,C5,
#          D5,A5,E5,D5,C5,A4,G4,E4,D4,C4]

#Prediction
predict = model_cnn_mfcc.predict(demo_mfcc)
p = np.argmax(predict, axis = 1)+1
print(p)

[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 7 3 3 3 3 3 3 3 7 3 1 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 5 3 3 3 5 3 3 3 3 3 3 3 3 3 3 3 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 6 4 6
 4 4 4 4 4 4 4 4 6 6 6 6 6 4 6 6 6 4 6 6 6 4 4 4 6 6 6 6 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 3 3 3 3 3 3 3 3 3 3 3
 3 3 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3 3 1 3 2 3 3 3 3 2 3 3 2 2 2 2 2 2 2 3
 6 3 2 4]


# Frequency Domain Magnitudes for C3

In [16]:
#As can be seen, the frequency oscillates between the frequency for C3 (~130Hz) and that of C2 (~65Hz)

loc_freq = []
for n in range(len(mytest_audio[1])//1024 - 1):
     loc_freq.append(np.argmax(abs(np.fft.fft((mytest_audio[1][1024*n:1024*(n+1)]).astype(float)))))

T = 1.0/16000.
N = 2048
xf = np.linspace(0.0, 1.0/(2.0*T), N//2)

princ_freq = [xf[x] for x in loc_freq]
print(princ_freq)

[132.94232649071358, 132.94232649071358, 132.94232649071358, 7874.877810361681, 132.94232649071358, 132.94232649071358, 132.94232649071358, 62.56109481915934, 62.56109481915934, 62.56109481915934, 62.56109481915934, 62.56109481915934, 62.56109481915934, 62.56109481915934, 7945.259042033235, 62.56109481915934, 7945.259042033235, 62.56109481915934, 62.56109481915934, 62.56109481915934, 62.56109481915934, 62.56109481915934]


#Testing on Digitally Produced C Scale at 120 BPM

In [31]:
from scipy.io.wavfile import read
path = '/content/drive/My Drive/Cscale_quantized_120bpm.wav'
y,s = librosa.core.load(path, sr = 16000)
demo_audio = np.array(y*32000)

In [ ]:
from IPython.display import Audio
Audio(demo_audio,rate=16000)

In [32]:
#Accurate to within .08 at 120BPM which is around a typical BPM seen in Rock or Pop music
#Only issue is being unable to discern two consecutive instances of the same note

demo_chroma = []
for n in range((len(demo_audio))//1024 - 1):
     demo_chroma.append(librosa.feature.chroma_stft(y=(demo_audio[1024*n:1024*(n+2)]).astype(float), sr=16000))
demo_chroma = np.array(demo_chroma).reshape(len(demo_chroma),12, 5)
demo_chroma = demo_chroma.reshape(demo_chroma.shape[0],12,5,1)


predict = model_cnn_chroma.predict(demo_chroma)

p = np.argmax(predict, axis = 1)
number+= len(p)
notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
pred_notes = []

#Weed out some obvious mistakes in prediction such as a note only being predicted once, which would be too quick to have actually been played
for idx, i in enumerate(p[:len(p)-1]):
  if i == p[idx+1] or i == p[idx-1]:
    pred_notes.append(notes[i])
  else:
    pred_notes.append(pred_notes[-1])
print(np.array(pred_notes))
bpm = 120
bps = bpm/60
classification_time = .064
curr_note = pred_notes[0]
count = 0
final_notes = []

#Find how many recurring instances of a note were perceived to find the length it was played
for note in pred_notes:
  if note == curr_note:
    count += 1
  else:
    beats = (count * classification_time) * bps
    final_notes.append(curr_note + ' ' + str(beats) + ' beats')
    count = 1
    curr_note = note
print(final_notes)

['C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'D' 'D' 'D'
 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'E' 'E' 'E' 'E' 'E'
 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F'
 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'G' 'G' 'G' 'G' 'G' 'G' 'G' 'G' 'G' 'G'
 'G' 'G' 'G' 'G' 'G' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C'
 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'A' 'A' 'A' 'A' 'A' 'A'
 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'G' 'G' 'G' 'G' 'G' 'G' 'G' 'G' 'G'
 'G' 'G' 'G' 'G' 'G' 'G' 'G' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F' 'F'
 'F' 'F' 'F' 'F' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E'
 'E' 'E' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D' 'D'
 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C